In [274]:
import numpy as np
import cplex
import random
import time
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [275]:
n_depot, n_vehicle = 10, 3
distance_matrix = pd.read_csv('distance_matrix.csv', names = [str(i) for i in range(n_depot)])
distance_matrix = np.array(distance_matrix)
orders = pd.read_csv('orders.csv')
initial_locations = [4, 10 ,2]
n_order = orders.shape[0] - len(initial_locations)
n_order_all = orders.shape[0]

In [276]:
class Subproblem():
    def __init__(self, n_dim):
        self.n_dim = n_dim
        self.variable_names = ['x' + str(i + 1) for i in range(self.n_dim)]
        self.constraints_rows, self.rhs  = self.set_constraints_byrows()
        self.n_constraints = len(self.constraints_rows)
        self.opt_solution = []
        self.cost_function_iterations = []
    
    def check_dim():
        assert(len(self.obj_coefficents) == len(self.variable_names))
        assert(len(self.obj_coefficents) == self.n_dim)
    
    def get_optvalues(self):
        self.obj_values = self.subproblem_cplex.solution.get_objective_value()
        print ("Solution value:   ",  self.obj_values)
        self.opt_solution = []
        for i, x in enumerate(self.subproblem_cplex.solution.get_values()):
            if i < self.n_dim:
                self.opt_solution.append(x)
    
    def set_lamda(self, lamda):
        self.lamda = lamda

    def get_solvable_status(self):
        status = self.subproblem_cplex.solution.get_status()
        if status == self.subproblem_cplex.solution.status.unbounded:
            print("Model is unbounded")
        if status == self.subproblem_cplex.solution.status.infeasible:
            print("Model is infeasible")
        if status == self.subproblem_cplex.solution.status.infeasible_or_unbounded:
            print("Model is infeasible or unbounded")
    
    def set_method(self, method):
        alg = self.subproblem_cplex.parameters.lpmethod.values

        if method == "o":  
            self.subproblem_cplex.parameters.lpmethod.set(alg.auto)
        elif method == "p":
            self.subproblem_cplex.parameters.lpmethod.set(alg.primal)
        elif method == "d":
            self.subproblem_cplex.parameters.lpmethod.set(alg.dual)
        elif method == "b":
            self.subproblem_cplex.parameters.lpmethod.set(alg.barrier)
            self.subproblem_cplex.parameters.barrier.crossover.set(self.Subproblem_LP_TU.parameters.parameters.barrier.crossover.values.none)
        elif method == "h":
            self.subproblem_cplex.parameters.lpmethod.set(alg.barrier)
        elif method == "s":
            self.subproblem_cplex.parameters.lpmethod.set(alg.sifting)
        elif method == "c":
            self.subproblem_cplex.parameters.lpmethod.set(alg.concurrent)
        else:
            raise ValueError('method must be one of "o", "p", "d", "b", "h", "s" or "c"')
    
    def solve(self, method = 'o'):
        self.subproblem_cplex = cplex.Cplex()
        self.subproblem_cplex.objective.set_sense(self.subproblem_cplex.objective.sense.maximize)
        lb, ub = self.set_lb_ub()
        self.obj_coefficents = self.set_obj()
        self.subproblem_cplex.variables.add(obj = self.obj_coefficents, lb = lb, ub = ub, names = self.variable_names)   
        
        self.senses = self.set_senses()
        self.rhs = self.set_rhs()
        row_names = ['c'+ str(i + 1) for i in range(self.n_constraints)]

        self.subproblem_cplex.linear_constraints.add(lin_expr = self.constraints_rows, senses = self.senses, rhs = self.rhs, names = row_names)
        self.set_method(method)
        self.subproblem_cplex.solve()
        self.get_solvable_status()
        self.get_optvalues()
        

In [277]:
class Subproblem_x(Subproblem):
    def __init__(self, n_dim):
        Subproblem.__init__(self, n_dim)
        self.big_m = 570.0
        
    def set_constraints_byrows(self):
        order_to_vehicle = self.compute_constraints_order_to_vehicle()
        initial_locations_to_vehicle = self.compute_constraints_initial_locations_to_vehicle()
        rhs = self.set_rhs()
        return order_to_vehicle + initial_locations_to_vehicle, rhs
       
    def compute_constraints_order_to_vehicle(self):
        order_to_vehicle = []
        for i in range(n_order):
            order_to_vehicle.append([self.variable_names[i: self.n_dim : n_order_all], [1] * n_vehicle])
        return order_to_vehicle
    
    def compute_constraints_initial_locations_to_vehicle(self):
        initial_locations_to_vehicle = []
        n_initial_location = n_order_all - n_order
        for i in range(n_initial_location):
            for j in range(n_initial_location):
                initial_locations_to_vehicle.append([[self.variable_names[i * n_order_all + j + n_order]], [1]])

        return initial_locations_to_vehicle
    
    def set_rhs(self):
        rhs = [1] * n_order
        rhs = rhs + [1, 0, 0, 0 ,1, 0, 0, 0, 1]
        return rhs
    
    def set_senses(self):
        senses = 'L'* n_order + 'E' * (n_order_all - n_order) * (n_order_all - n_order)
        return senses
    
    def set_obj(self):
        obj_coefficents = self.compute_obj_f1() + self.compute_obj_f2()
        return obj_coefficents
    
    def compute_obj_f1(self):
        self.orders_time = self.compute_orders_time()
        obj_coefficents_f1 = self.orders_time * n_vehicle
        return np.array(obj_coefficents_f1)
    
    def compute_orders_time(self):
        orders_time = []
        for i in range(n_order_all):
            orders_time.append(int(distance_matrix[orders['origin depot'][i] - 1][orders['destination depot'][i] - 1]))
        return orders_time
    
    def compute_obj_f2(self):
        obj_coefficents_f2 = np.zeros((n_vehicle, n_order_all))
        for i in range(n_order_all):
            for j in range(n_order_all):
                for k in range(n_vehicle):
                    if i!=j:
                        obj_coefficents_f2[k][i] -= self.lamda[i][j][k]
                        obj_coefficents_f2[k][j] -= self.lamda[i][j][k]
                        
        obj_coefficents_f2 = self.big_m * obj_coefficents_f2.reshape(self.n_dim)
        return obj_coefficents_f2
    
    def set_lb_ub(self):
        ub = [1] * self.n_dim
        lb = [0] * self.n_dim
        return lb, ub
                          
    def write_model(self):
        self.subproblem_cplex.write('Subproblem_x.lp')       
        
    def output_results(self):
        path = os.getcwd()
        path = path + '\\results\\'
        with open(path + 'Subproblem_x_result.txt', 'w') as f:
            for i in range(n_vehicle):
                f.write(str(self.opt_solution[i * n_order_all: (i + 1) * n_order_all]))
                f.write('\n')

In [278]:
class Subproblem_y():
    def __init__(self, n_dim):
        self.n_dim = n_dim
        self.big_m = 570.0
        self.obj_values = 0
        self.opt_solution = self.set_init_opt_solution()
    
    def compute_miu(self):
        miu = np.zeros((n_order, n_order))
        for i in range(n_order):
            for j in range(n_order):
                miu[i][j] = self.lamda[i][j][:].sum()
        return miu
    
    def set_init_opt_solution(self):
        opt_solution = np.zeros((n_order_all, n_order_all))
        opt_solution[n_order:n_order_all, 0:n_order] = np.ones((n_vehicle, 1))
        return opt_solution
    
    def set_lamda(self, lamda):
        self.lamda = lamda
        
    def solve(self):
        self.miu = self.compute_miu()
        for i in range(n_order):
            for j in range(n_order):
                if (i < j):
                    if (self.miu[i][j] >= self.miu[j][i]):
                        self.opt_solution[i][j], self.opt_solution[j][i] = 0, 1
                    else:
                        self.opt_solution[i][j], self.opt_solution[j][i] = 1, 0 
                        
    def compute_obj_values(self):    
        self.obj_values = 0
        for k in range(n_vehicle):
            for i in range(n_order_all):
                for j in range(n_order_all):
                    if i!=j:
                        self.obj_values -= self.lamda[i][j][k] * self.opt_solution[i][j]
        self.obj_values = self.obj_values * self.big_m
        return self.obj_values    
    
    def output_results(self):
        path = os.getcwd()
        path = path + '\\results\\'
        with open(path + 'Subproblem_y_result.txt', 'w') as f:
            for i in range(n_order_all):
                f.write(str(self.opt_solution[i:n_order_all: (i + 1)*n_order_all]))
                f.write('\n')

In [279]:
class Subproblem_time(Subproblem):
    def __init__(self, n_dim):
        Subproblem.__init__(self, n_dim)
        
    def set_constraints_byrows(self):
        order_to_vehicle = self.compute_constraints_origin_to_destination()
        rhs = self.set_rhs()
        return order_to_vehicle, rhs
       
    def compute_constraints_origin_to_destination(self):
        origin_to_destination = []
        for i in range(n_order_all):
            origin_to_destination.append([self.variable_names[i: self.n_dim: n_order_all], [-1, 1]])
        return origin_to_destination

    
    def set_rhs(self):
        orders_time = self.compute_orders_time()
        rhs = orders_time
        return rhs
    
    def compute_orders_time(self):
        orders_time = []
        for i in range(n_order_all):
            orders_time.append(int(distance_matrix[orders['origin depot'][i] - 1][orders['destination depot'][i] - 1]))
        return orders_time
    
    def set_senses(self):
        senses = 'E'* self.n_constraints
        return senses
    
    def set_lamda(self, lamda):
        self.lamda = lamda
        
    def set_obj(self):
        obj_coefficents = self.compute_obj_f5()
        return obj_coefficents
    
    def compute_obj_f5(self):
        obj_coefficents_f5 = np.zeros(self.n_dim)
        for i in range(n_order):
            for j in range(n_order):
                for k in range(n_vehicle):
                    if i!=j:
                        obj_coefficents_f5[i] += self.lamda[i][j][k]
                        obj_coefficents_f5[j] -= self.lamda[i][j][k]
                        
        return obj_coefficents_f5
    
    def set_lb_ub(self):
        ub = orders['deadline for pick up'].tolist() + [cplex.infinity] * n_order_all
        lb = [0] * self.n_dim
        return lb, ub
                        
    def write_model(self):
        self.subproblem_cplex.write('Subproblem_time.lp')       
    
    def output_results(self):
        path = os.getcwd()
        path = path + '\\results\\'
        with open(path + 'Subproblem_time_result.txt', 'w') as f:
            f.write(str(self.opt_solution))
            f.write('\n')

In [280]:
class Dual_Problem():
    def __init__(self):
        self.lamda = - np.zeros((n_order_all, n_order_all, n_vehicle))
        self.m, self.r, self.alpha = 5, 0.01, 1.0
        self.step_init, self.step, self.iteration_time = 0.0, 0.0, 0
        self.subgradients = - np.zeros((n_order_all, n_order_all, n_vehicle))
        self.obj_values_iterations = []
        self.subgradients_norm_iterations = []
        self.step_iterations = []
        self.obj_values = 0
        self.big_m = 570
    
    def init_subproblem_opt_solution(self, x, y, t):
        self.x, self.y, self.t = x, y, t
        
    def compute_orders_transfer_time(self):
        self.orders_transfer_time = np.zeros((n_order_all, n_order_all))
        for i in range(n_order_all):
            for j in range(n_order_all):
                 self.orders_transfer_time[i][j] = distance_matrix[orders['destination depot'][i] - 1][orders['origin depot'][j] - 1]
                
    def compute_subgradients(self):
        for i in range(n_order_all):
            for j in range(n_order_all):
                for k in range(n_vehicle):
                    if i!=j:
                        self.subgradients[i][j][k] = (self.t[i] + self.orders_transfer_time[i][j] -  self.t[j] + (3 - self.y[i][j] -  self.x[k*n_order_all+i] -  self.x[k*n_order_all+j])* self.big_m)
                    
    def compute_stepsize(self):
        subgradient_norm = np.linalg.norm(self.subgradients)**2
        assert(subgradient_norm != 0)
        self.estimation_opt_obj_values = 250
        self.step = (self.obj_values - self.estimation_opt_obj_values)/ subgradient_norm
        return self.step
    
    def update_lamda(self):
        self.lamda = self.lamda - self.step * self.subgradients
        self.lamda = np.minimum(self.lamda, 0)
        self.iteration_time += 1
        return self.lamda
    
    def compute_relaxed_problem_obj_values(self, orders_time):
        obj_values_original = 0
        self.obj_values = 0
        for j in range(n_order):
            for i in range(n_vehicle):
                self.obj_values += self.x[i * n_order_all + j] * orders_time[j]
        
        obj_values_original = self.obj_values
        for i in range(n_order_all):
            for j in range(n_order_all):
                for k in range(n_vehicle):
                    if i!=j:
                        self.obj_values += self.lamda[i][j][k] * (self.t[i] + self.orders_transfer_time[i][j] -  self.t[j] + (3 - self.y[i][j] -  self.x[k*n_order_all+i] -  self.x[k*n_order_all+j])* self.big_m)

        self.obj_values_iterations.append([self.obj_values, obj_values_original])
    
    def output_results(self):
        path = os.getcwd()
        path = path + '\\results\\'
        with open(path + 'lamda_result.txt', 'w') as f:
            for i in range(n_vehicle):
                f.write(str(self.lamda[:][:][i]))
                f.write('\n')
                f.write('\n')

In [281]:
subproblem_x = Subproblem_x(n_order_all * n_vehicle)
subproblem_y = Subproblem_y(((n_order + n_vehicle) * n_order))
subproblem_time = Subproblem_time(2 * n_order_all)

dual_problem = Dual_Problem()
dual_problem.compute_orders_transfer_time()
iteration_times = 20

for i in range(iteration_times):
    subproblem_x.set_lamda(dual_problem.lamda)
    subproblem_x.set_obj()
    subproblem_x.solve()
    
    subproblem_y.set_lamda(dual_problem.lamda)
    subproblem_y.solve()
    
    subproblem_time.set_lamda(dual_problem.lamda)
    subproblem_time.set_obj()
    subproblem_time.solve()
    
    dual_problem.init_subproblem_opt_solution(subproblem_x.opt_solution, subproblem_y.opt_solution, subproblem_time.opt_solution)
    dual_problem.compute_subgradients()
    dual_problem.compute_stepsize()
    dual_problem.update_lamda()
    dual_problem.compute_relaxed_problem_obj_values(subproblem_x.orders_time)

CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve eliminated 21 rows and 45 columns.
All rows and columns eliminated.
Presolve time = 0.00 sec. (0.01 ticks)
Solution value:    176.0
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve eliminated 15 rows and 30 columns.
All rows and columns eliminated.
Presolve time = 0.00 sec. (0.01 ticks)
Solution value:    0.0
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve eliminated 21 rows and 45 columns.
All rows and columns eliminated.
Presolve time = 0.00 sec. (0.01 ticks)
Solution value:    176.0
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve eliminated 15 rows and 30 columns.
All rows and columns eliminated.
Presolve time = 0.00 sec. (0.01 ticks)
Solution value:    0.0
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve eliminated 21 rows and 45 columns.
A

All rows and columns eliminated.
Presolve time = 0.00 sec. (0.01 ticks)
Solution value:    0.0
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve eliminated 21 rows and 45 columns.
All rows and columns eliminated.
Presolve time = 0.00 sec. (0.01 ticks)
Solution value:    176.0
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve eliminated 15 rows and 30 columns.
All rows and columns eliminated.
Presolve time = 0.00 sec. (0.01 ticks)
Solution value:    0.0


In [282]:
subproblem_x.write_model()
subproblem_x.output_results()
subproblem_time.write_model()
subproblem_time.output_results()
subproblem_y.output_results()
dual_problem.output_results()

In [283]:
print(dual_problem.obj_values_iterations)

[[176.0, 176.0], [176.0, 176.0], [176.0, 176.0], [176.0, 176.0], [176.0, 176.0], [176.0, 176.0], [176.0, 176.0], [176.0, 176.0], [176.0, 176.0], [176.0, 176.0], [176.0, 176.0], [176.0, 176.0], [176.0, 176.0], [176.0, 176.0], [176.0, 176.0], [176.0, 176.0], [176.0, 176.0], [176.0, 176.0], [176.0, 176.0], [176.0, 176.0]]
